In [1]:
import sys
import numpy as np
import pandas as pd
sys.path.append('../src/')

In [2]:
%load_ext autoreload
%autoreload 2

import bjorn_support as bs
import mutations as bm
import unsupervised_learning as bul

In [3]:
fp = '/Users/al/Documents/scripps/analysis/gisaid/mutations_2021-03-09.csv'
muts = pd.read_csv(fp)

/Users/al/anaconda3/envs/bjorn/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (7,51) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [68]:
muts = muts[muts['is_synonymous']==False]

In [125]:
lineage = 'B.1.526'
data = (muts.loc[muts['pangolin_lineage']==lineage, 'mutation']
        .value_counts()
        .to_frame()
        .reset_index()
        .rename(columns={'index': 'name', 'mutation': 'count'}))#.values
data['frequency'] = (data['count'] - np.min(data['count']))/np.ptp(data['count'])
data = data[data['frequency']>=0.05]
# data

In [126]:
from scipy.spatial import distance
from scipy.stats import mode
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import SpectralClustering
from sklearn.metrics import confusion_matrix, accuracy_score
from skimage.io import imread

In [127]:
def kmeans_unsupervised(X:np.ndarray, k:int, centroids=None, tolerance=1e-2):
    """K-Means Clustering with unsupervised learning trick to infer characteristic mutations
    for a given SARS-CoV-2 lineage"""
    # Initialize Input, Centroids, Clusters
    if len(X.shape) < 2: X = np.expand_dims(X, axis=1)
    if centroids: 
        centroids = bul.init_centroids(X, k)
    else:
        centroids = X[np.random.choice(X.shape[0], size=k, replace=False), :]
    clusters = [[] for c in centroids]
    # Until Convergence
    while(True):
        # Centroid Norm at t-1
        old_centroids_norm = np.linalg.norm(centroids)
        # Minimum Distance to Centroid Per Record
        js = np.argmin(distance.cdist(X, centroids), axis=1)
        # Assign Records to Clusters
        for i, c in enumerate(clusters):
             clusters[i] = list(np.where(js == i)[0])
        # Recompute Cluster Centroids
        for j, c in enumerate(centroids):
            centroids[j] = np.mean(X[clusters[j]], axis=0)
        # Centroid Norm at t
        centroids_norm = np.linalg.norm(centroids)
        # Convergence!
        if np.abs(centroids_norm - old_centroids_norm) < tolerance:
            preds = np.zeros_like(X)
            clusters = np.asarray(clusters)[np.argsort(centroids.squeeze())[::-1]]
#             return clusters, centroids
            for i, c in enumerate(clusters):
                preds[c] = i
            return preds.squeeze()

In [128]:
X = np.expand_dims(data['frequency'].to_numpy(), axis=1)
preds = kmeans_unsupervised(X, k=2, centroids="kmeans++") == 0
preds

/Users/al/anaconda3/envs/bjorn/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False, False, False,
       False, False, False, False, False, False])

In [129]:
# data['is_characteristic'] = bul.spectral_clustering_unsupervised(X, num_classes=2, n_trees=100)

In [130]:
data['is_characteristic'] = preds
data['lineage'] = lineage
data.to_csv(f'/Users/al/Documents/scripps/analysis/bjorn/{lineage}_char_mutations.csv', index=False)

In [131]:
data

,name,count,frequency,is_characteristic,lineage
0,S:D614G,1621,1.000000,True,B.1.526
1,S:D253G,1620,0.999383,True,B.1.526
2,ORF1b:Q1011H,1618,0.998148,True,B.1.526
3,ORF1b:P314L,1617,0.997531,True,B.1.526
4,ORF1a:T265I,1611,0.993827,True,B.1.526
5,ORF1a:L3201P,1606,0.990741,True,B.1.526
6,S:T95I,1602,0.988272,True,B.1.526
7,ORF3a:Q57H,1591,0.981481,True,B.1.526
8,ORF1a:DEL3675/3677,1574,0.970988,True,B.1.526
9,S:L5F,1561,0.962963,True,B.1.526
